In [1]:
# Written by Chongbin Zhang. 
# Designed to facilitate UCSD HDH's calendar import from events organized in excel table. 

# necessary imports
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from datetime import datetime
from icalendar import Calendar, Event

In [50]:
def stringIsInt(strin):
    if type(strin) is not str:
        raise(InputIsNotStringError())
    try:
        intout = int(strin)
        return intout
    except ValueError:
        return False

In [54]:
# open the excel and parse all the columns into lists, df stands for data frame
df = pd.read_excel('ERCDeskEvents.xls', sheet_name='Sheet1')
listConfName = df['Conference Name']
listHousArea = df['College Housing Area']
listConfCord = df['Conference Coordinator']
listArrvDate = df['Arrival 1 : Date']
listArrTm_Ct = df['Arrival 1 : Time, Count']
listChkInstr = df['Arrival 1 : Check-In Instructions']
listMobi_Plc = df['Mobile Check-In Location  & Date']
    
listArrvTime = listArrCount = []

for i in range(0, 150): # split arrival time & people count, 0~150 is ERC Desk
    temp = listArrTm_Ct[i].split(', ')
    if temp[0].find(':') == -1 and temp[0].find('a') == -1 and temp[0].find('p') == -1:
        print("ERROR parsing: ", listConfName[i], ", because arrival time count is entered incorrectly as:", 
              temp[0], "Please enter this event manually. ")
    else:
        listArrvTime.append(temp[0])
    if len(temp) <= 1:
        listArrCount.append(0)
    else:
        income = stringIsInt(temp[1])
        if not income:
            print("ERROR parsing: ", listConfName[i], ", because people count is entered incorrectly as:", 
                  temp[1], ". Please enter this event manually. ")
        else:
            listArrCount.append(income)

listArrTm_Ct = None # free memory
cal = Calendar()

ERROR parsing:  Alexa Café by iD Tech Camps-Week 2 , because people count is entered incorrectly as: 0 . Please enter this event manually. 
ERROR parsing:  Alexa Café by iD Tech Camps-Week 3 , because people count is entered incorrectly as: 0 . Please enter this event manually. 
ERROR parsing:  SKYUS-Senri High School , because arrival time count is entered incorrectly as: Check in time TBD Please enter this event manually. 
ERROR parsing:  SKYUS-Senri High School , because people count is entered incorrectly as: 44 Participants including chaperones . Please enter this event manually. 
ERROR parsing:  TCLI Summer Leadership Program , because people count is entered incorrectly as: 14 Staff only on this date . Please enter this event manually. 
ERROR parsing:  SKYUS-Senri High School , because arrival time count is entered incorrectly as: Check out by 11AM Please enter this event manually. 
ERROR parsing:  SKYUS-Senri High School , because people count is entered incorrectly as: 44 Part

In [ ]:
# iterate through the table to create .ics file, 150 is the last for ERC desk in this file
for i in range(3, 6):
    print('conference name: ', listConfName[i])
    event = Event()
        
    # date start & end
    peopleCount = listArrTm_Ct[i][1]
    
    try: 
        separator = listArrTm_Ct[i][0].index('-') # has a end time
        beginTime = listArrTm_Ct[i][0][0:separator].split(':')
        endTime = listArrTm_Ct[i][0][separator + 1:len(listArrTm_Ct[i][0]) + 1].split(':')
    except ValueError: # no end time b/c no '-' in arrival time: assume 1 hour long 
        begin = listArrTm_Ct[i][0].index('m')
        beginTime = listArrTm_Ct[i][0][0:(begin + 1)]
        beginTime = beginTime.split(':')
        endHour = int(beginTime[0:beginTime.index(':')]) + 1
        endTime = str(endHour) + ':' + beginTime[beginTime.index(':') + 1:len(beginTime) + 1]
            
    print('begin time: ', beginTime)
    begin = datetime(int(listArrvDate[i].year), int(listArrvDate[i].month), int(listArrvDate[i].day), 
                     int(beginTime[0]), int(beginTime[1][0:2]))
    end = datetime(int(listArrvDate[i].year), int(listArrvDate[i].month), int(listArrvDate[i].day), 
                   int(endTime[0]), int(endTime[1][0:2]))
    
    # Add people count in title
    summary = listConfName[i] + ' (' + str(peopleCount) + ')'
    
    # categorize
    if listMobi_Plc[i] == "nan": 
        summary = summary + ' (Check-in)'
    # elif (font is red): 
    #     event.add('categories, 'Check-out')
    else: 
        summary += ' (Mobile Check-in)'
    
    event.add('summary', summary)
    event.add('description', listChkInstr[i])
    event.add('dtstart', begin)
    event.add('dtend', end)
    cal.add_component(event)

    filename = './' + summary + '.ics'
    file = open(filename, 'wb')
    file.write(cal.to_ical())
    file.close()


